# Model Evaluation: Quick Start Guide

Unlike traditional software systems that rely on prescribed specifications and application code, AI systems based on machine learning models depend on training data to map inputs to outputs. Consequently, these systems are inherently non-deterministic and may produce errors due to variability in the training data or the probabilistic nature of the underlying algorithms. To evaluate such systems, benchmarks are commonly used to address user concerns, such as accuracy and bias. However, since benchmarks can be manipulated over time to achieve favorable results, it is essential to establish a flexible evaluation framework that supports rapid updates to evaluation criteria and benchmark selection. Given the critical role of testing and evaluation in deploying AI systems, there is a pressing need for a consistent methodology and robust tool support for these activities. 

In the context of generative AI, evaluation serves to provide evidence that fosters user trust in models and systems. Specifically, it involves measuring and quantifying how a model or system responds to inputs. Are the responses within acceptable bounds—free from hate speech, hallucinations, or other harmful outputs? Are they useful, cost-effective, and reliable?

[Unitxt](https://www.unitxt.ai) is an innovative library for textual data preparation and evaluation of generative language models. It deconstructs the data preparation and evaluation flows into modular components, enabling easy customization and sharing between practitioners. Unitxt as an extensive [catalog](https://www.unitxt.ai/en/latest/catalog/catalog.__dir__.html) of datasets, tasks, templates, metrics and textual operators, that can be used as-is, or be combined in innovative ways to create new evaluations.

In this example, lets evaluate Granite's performance on the [OpenbookQA](https://huggingface.co/datasets/allenai/openbookqa) dataset from HuggingFace. We load the dataset using its card, generate the model client, run inference, and evaluate the results.

_Note_: To get set up with Replicate, see [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb), then get an API token [here](https://replicate.com/account/api-tokens). 

## Load Dependencies

In [ ]:
%pip install replicate
%pip install unitxt
%pip install openai
%pip install litellm
%pip install diskcache
%pip install tenacity
%pip install tabulate
%pip install git+https://github.com/ibm-granite-community/utils

and

In [ ]:
from unitxt.api import evaluate, load_dataset
from unitxt.inference import CrossProviderInferenceEngine

from ibm_granite_community.notebook_utils import get_env_var

import nest_asyncio
nest_asyncio.apply()

## Load the dataset from the Unitxt catalog

Since PopQA is one of hundreds of datasets already contributed to Unitxt, we can simply load it from the catalog. Its [task card](https://www.unitxt.ai/en/latest/catalog/catalog.cards.openbook_qa.html) captures all the necessary details on how to load, transform, format, and evaluate the dataset.

In this example, we keep all the default templates and formats, but limit loading to 20 prompts to reduce inference time and cost.

In [ ]:
dataset = load_dataset(card="cards.openbook_qa",split="test", loader_limit=20)

## Instantiate the evaluation client

Unitxt supports a wide range of inference engines for running models locally or remotely. For a complete list, see the [catalog](https://www.unitxt.ai/en/latest/catalog/catalog.engines.__dir__.html), or follow the [tutorial](https://www.unitxt.ai/en/latest/docs/inference.html).

In this example, we are using a [CrossProviderInferenceEngine](https://www.unitxt.ai/en/latest/unitxt.inference.html#unitxt.inference.CrossProviderInferenceEngine) inference engine that wraps popular APIs for providers such as watsonx, openai, azure, aws, and many more.

In [ ]:
model = CrossProviderInferenceEngine(
    model="replicate/ibm-granite/granite-3.3-8b-instruct",
    provider="replicate",
    credentials={'api_token': get_env_var('REPLICATE_API_TOKEN')},
    provider_specific_args={'max_requests_per_second': 1}
)

## Generate predictions

In [ ]:
predictions = model(dataset)

## Evaluate the predictions to determine results

In [ ]:
results = evaluate(predictions=predictions, data=dataset)

## Print the scores

In [ ]:
print("Global Results:")
print(results.global_scores.summary)

print("Instance Results:")
print(results.instance_scores.summary)